## Calculate Performance Matrices for AEMO Forecasted Demand

#### Load datasets

* Prepared Demand data file
* Prepared AEMO forecast demand file

In [43]:
import pandas as pd
import datetime as dt
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error

In [44]:
td_df = pd.read_csv("../data/final_data_2010_2021.csv")
td_df = td_df[['DATETIME','TOTALDEMAND']]
aemo_fd_df = pd.read_csv("../data/forecastdemand_nsw.csv")
print(td_df)
print(aemo_fd_df)

                   DATETIME  TOTALDEMAND
0       2010-01-01 00:00:00      8038.00
1       2010-01-01 00:30:00      7809.31
2       2010-01-01 01:00:00      7483.69
3       2010-01-01 01:30:00      7117.23
4       2010-01-01 02:00:00      6812.03
...                     ...          ...
196460  2021-03-16 22:00:00      7373.83
196461  2021-03-16 22:30:00      7345.78
196462  2021-03-16 23:00:00      7218.99
196463  2021-03-16 23:30:00      7056.88
196464  2021-03-17 00:00:00      6999.23

[196465 rows x 2 columns]
                  DATETIME  FORECASTDEMAND
0      2018-01-01 00:00:00     6599.958028
1      2018-01-01 00:30:00     6462.985139
2      2018-01-01 01:00:00     6302.716986
3      2018-01-01 01:30:00     6066.275000
4      2018-01-01 02:00:00     5865.197600
...                    ...             ...
56252  2021-03-17 22:00:00     7317.702687
56253  2021-03-17 22:30:00     7266.887647
56254  2021-03-17 23:00:00     7173.866522
56255  2021-03-17 23:30:00     7043.236857
56256  2

#### Set analysis date range

In [45]:
f_start_date = dt.date(2020, 7, 24) 
f_end_date = dt.date(2021, 2, 28)

#### Filter datasets to date range used for analysis

In [46]:
td_df['DATETIME'] = pd.to_datetime(td_df['DATETIME'], format='%Y-%m-%d  %H:%M:%S')
aemo_fd_df['DATETIME'] = pd.to_datetime(aemo_fd_df['DATETIME'], format='%Y-%m-%d  %H:%M:%S')

td_df = td_df[(td_df['DATETIME'].dt.date >= f_start_date) & (td_df['DATETIME'].dt.date <= f_end_date)]
aemo_fd_df = aemo_fd_df[(aemo_fd_df['DATETIME'].dt.date >= f_start_date) & (aemo_fd_df['DATETIME'].dt.date <= f_end_date)]

print(td_df)
print(aemo_fd_df)

                  DATETIME  TOTALDEMAND
185136 2020-07-24 00:00:00      8799.44
185137 2020-07-24 00:30:00      8664.64
185138 2020-07-24 01:00:00      8394.84
185139 2020-07-24 01:30:00      8178.87
185140 2020-07-24 02:00:00      7975.65
...                    ...          ...
195691 2021-02-28 21:30:00      7870.47
195692 2021-02-28 22:00:00      7675.40
195693 2021-02-28 22:30:00      7527.73
195694 2021-02-28 23:00:00      7301.00
195695 2021-02-28 23:30:00      7163.21

[10560 rows x 2 columns]
                 DATETIME  FORECASTDEMAND
44880 2020-07-24 00:00:00     8799.931127
44881 2020-07-24 00:30:00     8607.057917
44882 2020-07-24 01:00:00     8411.563151
44883 2020-07-24 01:30:00     8182.821486
44884 2020-07-24 02:00:00     7916.211333
...                   ...             ...
55435 2021-02-28 21:30:00     7853.082879
55436 2021-02-28 22:00:00     7657.957910
55437 2021-02-28 22:30:00     7479.415882
55438 2021-02-28 23:00:00     7268.718116
55439 2021-02-28 23:30:00     70

#### Merge the two datasets

In [47]:
full_data = pd.merge(td_df, aemo_fd_df, on="DATETIME", how='inner')
full_data.rename(columns={"TOTALDEMAND":"Actual Demand","FORECASTDEMAND":"AEMO Forecast Demand"},inplace=True)
print(full_data)

                 DATETIME  Actual Demand  AEMO Forecast Demand
0     2020-07-24 00:00:00        8799.44           8799.931127
1     2020-07-24 00:30:00        8664.64           8607.057917
2     2020-07-24 01:00:00        8394.84           8411.563151
3     2020-07-24 01:30:00        8178.87           8182.821486
4     2020-07-24 02:00:00        7975.65           7916.211333
...                   ...            ...                   ...
10555 2021-02-28 21:30:00        7870.47           7853.082879
10556 2021-02-28 22:00:00        7675.40           7657.957910
10557 2021-02-28 22:30:00        7527.73           7479.415882
10558 2021-02-28 23:00:00        7301.00           7268.718116
10559 2021-02-28 23:30:00        7163.21           7054.821714

[10560 rows x 3 columns]


#### Calaculate Performance Matrices

In [48]:
AEMO_rmse = round(np.sqrt(mean_squared_error(full_data["Actual Demand"],full_data["AEMO Forecast Demand"])),2)
AEMO_mae = round(mean_absolute_error(full_data["Actual Demand"],full_data["AEMO Forecast Demand"]),2)
AEMO_mape = round(mean_absolute_percentage_error(full_data["Actual Demand"],full_data["AEMO Forecast Demand"]),2)

print(f"RMSE: {AEMO_rmse}")
print(f"MAE: {AEMO_mae}")
print(f"MAPE: {AEMO_mape*100}%")

RMSE: 232.33
MAE: 160.01
MAPE: 2.0%
